In [1]:
import tensorflow as tf

In [2]:
tf.__version__

'2.6.2'

In [3]:
image_width = 227
image_height = 227
image_channel = 3
NUM_CLASSES = 2

In [4]:
# In this block we will define Alexnet

def Alexnet():
    model = tf.keras.Sequential([
        # layer 1
        tf.keras.layers.Conv2D(filters=96,
                               kernel_size=(11,11),
                               strides=4,
                               padding="valid",
                               activation=tf.keras.activations.relu,
                               input_shape=(image_height,image_width,image_channel)),
        
        tf.keras.layers.MaxPool2D(pool_size=(3,3),
                                  strides=2,
                                  padding="valid"),
        
        tf.keras.layers.BatchNormalization(),

        # layer 2
        tf.keras.layers.Conv2D(filters=256,
                               kernel_size=(5,5),
                               strides=1,padding="same",
                               activation=tf.keras.activations.relu),
        tf.keras.layers.MaxPool2D(pool_size=(3,3),
                                  strides=2,
                                  padding="same"),
        tf.keras.layers.BatchNormalization(),
        
        # layer 3
        tf.keras.layers.Conv2D(filters=384,
                               kernel_size=(3,3),
                               strides=1,
                               padding="same",
                               activation=tf.keras.activations.relu),
        
        # layer 4
        tf.keras.layers.Conv2D(filters=256,
                               kernel_size=(3,3),
                               strides=1,
                               padding="same",
                               activation=tf.keras.activations.relu),
        
        # layer 5
        tf.keras.layers.Conv2D(filters=256,
                               kernel_size=(3,3),
                               strides=1,
                               padding="same",
                               activation=tf.keras.activations.relu),
        
        tf.keras.layers.MaxPool2D(pool_size=(3,3),
                           strides=1,
                           padding="same"),
        tf.keras.layers.BatchNormalization(),
        
        # layer 6
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(units=4096,
                              activation=tf.keras.activations.relu),
        tf.keras.layers.Dropout(0.2),
        
        # layer 7
        tf.keras.layers.Dense(units=4096,
                              activation=tf.keras.activations.relu),
        tf.keras.layers.Dropout(0.2),
        
        # layer 8
        tf.keras.layers.Dense(units=NUM_CLASSES,
                              activation=tf.keras.activations.softmax)
        ])
    return model

In [5]:
# Preparation of dataset

import tensorflow as tf


def make_dataset():
    
    training_dir = "/home/vignesh/fish/fishes/train/" 
    test_dir = "/home/vignesh/fish/fishes/test/"
    validation_dir = "/home/vignesh/fish/fishes/validation/"
    image_width = 227
    image_height = 227
    image_channel = 3
    BATCH_SIZE = 8
    train_data_gen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1.0/255.0)
    training_generator = train_data_gen.flow_from_directory(training_dir,
                                                        target_size=(image_height,image_width),
                                                        batch_size=BATCH_SIZE,
                                                         seed = 7,
                                                         shuffle = True,
                                                         class_mode = "categorical")
    validation_data_gen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1.0/255.0)
    validation_generator = validation_data_gen.flow_from_directory(validation_dir,
                                                            target_size=(image_height,image_width),
                                                             color_mode="rgb",
                                                             batch_size = BATCH_SIZE,
                                                             seed = 7,
                                                             shuffle = True,
                                                             class_mode = "categorical"
                                                            )

    testing_data_gen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1.0/255.0)
    testing_generator = testing_data_gen.flow_from_directory(test_dir,
                                                             target_size=(image_height,image_width),
                                                             color_mode = "rgb",
                                                             batch_size = BATCH_SIZE,
                                                             seed = 7,
                                                             shuffle = True,
                                                             class_mode = "categorical")
  

    train_num = training_generator.samples
    valid_num = validation_generator.samples
    test_num = testing_generator.samples


    return training_generator, \
           validation_generator, \
           testing_generator, \
           train_num, valid_num, test_num


In [6]:
import tensorflow as tf

############################################################
# required configurations

NUM_CLASSES = 2
BATCH_SIZE = 8
EPOCHS = 2
model_dir = "image_classification_model.h5"
# training_dir ="/home/vignesh/fish/fishes/train"
# testing_dir = "/home/vignesh/fish/fishes/test"
# validation_dir = "/home/vignesh/fish/fishes/validation"
###########################################################

def get_model():
    model = Alexnet()
    model.compile(loss=tf.keras.losses.categorical_crossentropy,
                  optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
                  metrics=['accuracy'])
    return model


gpus = tf.config.experimental.list_physical_devices('GPU')

if gpus:
    for gpu in gpus:
        tf.config.experimental.set_memory_growth(gpu, True)

training_generator, validation_generator, testing_generator,train_num, valid_num, test_num = make_dataset()

# Use command tensorboard --logdir "log" to start tensorboard
tensorboard = tf.keras.callbacks.TensorBoard(log_dir='log')
callback_list = [tensorboard]

model = get_model()
model.summary()

# start training
model.fit(training_generator,epochs=EPOCHS,steps_per_epoch=train_num//BATCH_SIZE,callbacks=callback_list, validation_data=validation_generator)

# save the whole model
model.save(model_dir)



Found 1929 images belonging to 2 classes.
Found 57 images belonging to 2 classes.
Found 106 images belonging to 2 classes.
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 55, 55, 96)        34944     
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 27, 27, 96)        0         
_________________________________________________________________
batch_normalization (BatchNo (None, 27, 27, 96)        384       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 27, 27, 256)       614656    
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 14, 14, 256)       0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 14, 14, 256)       1024      